# 2/10/17 - Training RNNs as Fast as CNNs

- http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://arxiv.org/abs/1709.02755
- https://github.com/taolei87/sru
- https://arxiv.org/abs/1611.01576

### Implementation of standard RNN from scratch

- http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/
- http://www.wildml.com/2015/10/recurrent-neural-network-tutorial-part-4-implementing-a-grulstm-rnn-with-python-and-theano/
- https://gist.github.com/karpathy/d4dee566867f8291f086

$x_t$ is input at time step $t$

$h_t$ is hidden state at time step $t$. It is calculated via $h_t=f(Ux_t+Wh_{t-1})$, where $f$ is a non-linearity. $h_{0}$ (the initial hidden state) is typically initialised to all zeros.
  
$y_t$ is the output at step $t$, i.e. $y_t = V h_t$

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math


class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.w_xh = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to hidden, aka U
        self.w_hh = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to hidden, aka W
        self.w_hy = nn.Parameter(torch.Tensor(input_size, hidden_size)) #hidden to output, aka V
        
        self.b_h = nn.Parameter(torch.Tensor(hidden_size)) #hidden bias
        self.b_y = nn.Parameter(torch.Tensor(input_size)) #output bias
        
    def forward(self, x, h):
        
        #h_t = tanh(w_xh * x_t + w_hh * h_(t-1) + b_h) [note, here '*' implies matrix multiplication]
        h = F.tanh(torch.matmul(self.w_xh, x) + torch.matmul(self.w_hh, h) + self.b_h)
        
        #y_t = w_hy * h_t + b_y
        y = torch.matmul(self.w_hy, h) + self.b_y
        
        return y, h

In [8]:
#test to see if it goes through with no error
rnn = RNN(10, 100)

x = Variable(torch.Tensor(10))
h = Variable(torch.Tensor(100))

y, h = rnn(x, h)

### Implementation of Gated Recurrent Unit (GRU) from scratch
- The "new gate" terminology comes from http://pytorch.org/docs/master/nn.html#torch.nn.GRU
- https://arxiv.org/pdf/1412.3555.pdf
- https://arxiv.org/ftp/arxiv/papers/1701/1701.05923.pdf

In [9]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRU, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.w_xz = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to update gate
        self.w_hz = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to update gate
        self.w_xr = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to reset gate
        self.w_hr = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to reset gate
        self.w_xn = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to new gate
        self.w_hn = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to new gate
        self.w_hy = nn.Parameter(torch.Tensor(input_size, hidden_size)) #hidden to output gate
        
        self.b_z = nn.Parameter(torch.Tensor(hidden_size)) #update gate bias
        self.b_r = nn.Parameter(torch.Tensor(hidden_size)) #reset gate bias
        self.b_n = nn.Parameter(torch.Tensor(hidden_size)) #new gate bias
        self.b_y = nn.Parameter(torch.Tensor(input_size)) #output bia
        
    def forward(self, x, h):
        
        #z_t = sigmoid(w_xz * x_t + w_hz * h + b_z) [note, here '*' implies matrix multiplication]
        z = F.sigmoid(torch.matmul(self.w_xz, x) + torch.matmul(self.w_hz, h) + self.b_z)
        
        #r_t = sigmoid(w_xr * x_t + w_hr * h + b_r)
        r = F.sigmoid(torch.matmul(self.w_xr, x) + torch.matmul(self.w_hr, h) + self.b_r)
        
        #n_t = tanh(w_xn * x_t + w_hn * (r • h_(t-1)) + b_n) [note, here '•' is elementwise multiplication]
        n = F.tanh(torch.matmul(self.w_xn, x) + torch.matmul(self.w_hn, r * h) + self.b_n)
        
        #h_t = (1 - z_t) • n_t + z_t • h_t
        h = (1 - z) * n + z * h
        
        #y_t = w_hy * h_t + b_y
        y = torch.matmul(self.w_hy, h) + self.b_y
        
        return y, h

In [10]:
gru = GRU(10, 100)

x = Variable(torch.Tensor(10))
h = Variable(torch.Tensor(100))

y, h = gru(x, h)

params = gru.parameters()

id(y)

#for p in params:
 #   print(Variable(p.data))

140052903005264

### Implementation of Long Short-Term Memory (LSTM) from scratch
- https://apaszke.github.io/lstm-explained.html
- https://deeplearning4j.org/lstm.html

In [11]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.w_xf = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to forget gate
        self.w_hf = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to forget gate
        self.w_xi = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to input gate
        self.w_hi = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to forget gate
        self.w_xo = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to output gate
        self.w_ho = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to output gate
        self.w_xc = nn.Parameter(torch.Tensor(hidden_size, input_size)) #input to cell
        self.w_hc = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) #hidden to cell
        self.w_hy = nn.Parameter(torch.Tensor(input_size, hidden_size)) #hidden to output
                
        self.b_f = nn.Parameter(torch.Tensor(hidden_size)) #forget gate bias
        self.b_i = nn.Parameter(torch.Tensor(hidden_size)) #input gate bias
        self.b_o = nn.Parameter(torch.Tensor(hidden_size)) #output gate bias
        self.b_c = nn.Parameter(torch.Tensor(hidden_size)) #cell bias
        self.b_y = nn.Parameter(torch.Tensor(input_size)) #output bias
        
    def forward(self, x, h, c):
        
        #f_t = sigmoid(w_xf * x_t + w_hf * h_(t-1) + b_f) [note, here '*' implies matrix multiplication]
        f = F.sigmoid(torch.matmul(self.w_xf, x) + torch.matmul(self.w_hf, h) + self.b_f)

        #i_t = sigmoid(w_xi * x_t + w_hi * h_(t-1) + b_i)
        i = F.sigmoid(torch.matmul(self.w_xi, x) + torch.matmul(self.w_hi, h) + self.b_i)

        #o_t = sigmoid(w_xo * x_t + w_ho * h_(t-1) + b_o)
        o = F.sigmoid(torch.matmul(self.w_xo, x) + torch.matmul(self.w_ho, h) + self.b_o)

        #c_in = tanh(w_xc * x_t + w_hc * h_(t-1) + b_c)
        #'c_in' is also known as 'g' in the PyTorch docs (http://pytorch.org/docs/master/nn.html#torch.nn.LSTM)
        c_in = F.tanh(torch.matmul(self.w_xc, x) + torch.matmul(self.w_hc, h) + self.b_c) 

        #c_t = f_t • c_(t-1) + i_t • c_in [note, here '•' is elementwise multiplication]
        c = f * c + i * c_in
        
        #h_t = o_t • tanh(c_t)
        h = o * F.tanh(c)
        
        #y_t = w_hy * h_t + b_y
        y = torch.matmul(self.w_hy, h) + self.b_y
        
        return y, h, c

In [12]:
lstm = LSTM(10, 100)

x = Variable(torch.Tensor(10))
h = Variable(torch.Tensor(100))
c = Variable(torch.Tensor(100))

y, h, c = lstm(x, h, c)

### Implementation of Simple Recurrent Unit (SRU) from scratch
- https://arxiv.org/pdf/1709.02755.pdf
- https://github.com/taolei87/sru
- https://github.com/musyoku/chainer-sru
- https://github.com/titu1994/keras-SRU

$\tilde{x_t}=Wx$

$f_t = \sigma(W_fx_t+b_f)$

$r_t = \sigma(W_rx_t+b_r)$

$c_t = f_t \odot c_{t-1} + (1-f_t) \odot \tilde{x_t}$

$h_t = r_t \odot g(c_t) + (1-r_t) \odot x_t$

In [13]:
class SRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SRU, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.w_xt = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_xf = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_xr = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_hy = nn.Parameter(torch.Tensor(input_size, hidden_size))
        
        self.b_x = nn.Parameter(torch.Tensor(hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))
        self.b_r = nn.Parameter(torch.Tensor(hidden_size))
        self.b_y = nn.Parameter(torch.Tensor(input_size))
        
    def forward(self, x, h, c):
        
        t = torch.matmul(self.w_xt, x)
        
        f = F.sigmoid(torch.matmul(self.w_xf, x) + self.b_f)
        
        r = F.sigmoid(torch.matmul(self.w_xr, x) + self.b_r)

        c = f * c + (1 - f) * t

        h = r * F.tanh(c) + (1 - r) * x
        
        #y_t = w_hy * h_t + b_y
        y = torch.matmul(self.w_hy, h) + self.b_y
        
        return y, h, c

**NOTE:** SRU usage is slightly different as input dimensions have to be the same dimensions as the hidden dimensons. 

This means you need an embedding layer if input dim != hidden dim, which it usually will be.

In [14]:
sru = SRU(100, 100) #input and hidden dim must be equal!

x = Variable(torch.Tensor(100)) #input dim now 100, previously has been 10
h = Variable(torch.Tensor(100))
c = Variable(torch.Tensor(100))

y, h, c = sru(x, h, c)

In [15]:
# RE-DO BELOW WITH EMBEDDING AS SEPARATE CLASS AND FEED IN SINGLE EXAMPLE AT A TIME

In [16]:
"""class SRUWithEmbedding(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SRUWithEmbedding, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        self.w_xt = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_xf = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_xr = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_hy = nn.Parameter(torch.Tensor(input_size, hidden_size))
        
        self.b_x = nn.Parameter(torch.Tensor(hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))
        self.b_r = nn.Parameter(torch.Tensor(hidden_size))
        self.b_y = nn.Parameter(torch.Tensor(input_size))
        
    def forward(self, x, h, c):
        
        x = self.embedding(x)
        t = torch.matmul(self.w_xt, x)
        
        f = F.sigmoid(torch.matmul(self.w_xf, x) + self.b_f)
        
        r = F.sigmoid(torch.matmul(self.w_xr, x) + self.b_r)

        c = f * c + (1 - f) * t

        print("r",r.size())
        print("c",c.size())
        print("x",x.size())
        h = r * F.tanh(c) + (1 - r) * x
        
        #y_t = w_hy * h_t + b_y
        y = torch.matmul(self.w_hy, h) + self.b_y 
        
        return y, h, c"""

'class SRUWithEmbedding(nn.Module):\n    def __init__(self, input_size, hidden_size):\n        super(SRUWithEmbedding, self).__init__()\n        \n        self.input_size = input_size\n        self.hidden_size = hidden_size\n        \n        self.embedding = nn.Embedding(input_size, hidden_size)\n        \n        self.w_xt = nn.Parameter(torch.Tensor(hidden_size, input_size))\n        self.w_xf = nn.Parameter(torch.Tensor(hidden_size, input_size))\n        self.w_xr = nn.Parameter(torch.Tensor(hidden_size, input_size))\n        self.w_hy = nn.Parameter(torch.Tensor(input_size, hidden_size))\n        \n        self.b_x = nn.Parameter(torch.Tensor(hidden_size))\n        self.b_f = nn.Parameter(torch.Tensor(hidden_size))\n        self.b_r = nn.Parameter(torch.Tensor(hidden_size))\n        self.b_y = nn.Parameter(torch.Tensor(input_size))\n        \n    def forward(self, x, h, c):\n        \n        x = self.embedding(x)\n        t = torch.matmul(self.w_xt, x)\n        \n        f = F.si

In [17]:
"""sru = SRUWithEmbedding(1, 100) #input and hidden dim must be equal!

x = Variable(torch.ones([1]).long()) #input dim now 100, previously has been 10
h = Variable(torch.Tensor(100))
c = Variable(torch.Tensor(100))
print(x)
y, h, c = sru(x, h, c)"""

'sru = SRUWithEmbedding(1, 100) #input and hidden dim must be equal!\n\nx = Variable(torch.ones([1]).long()) #input dim now 100, previously has been 10\nh = Variable(torch.Tensor(100))\nc = Variable(torch.Tensor(100))\nprint(x)\ny, h, c = sru(x, h, c)'

### Implementation of Quasi-Recurrent Neural Network (QRNN) from scratch
- https://arxiv.org/abs/1611.01576
- https://github.com/JayParks/quasi-rnn
- https://github.com/salesforce/pytorch-qrnn

In [137]:
class QRNN(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size):
        super(QRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.kernel_size = kernel_size
        
        self.z_conv = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=kernel_size)
        self.f_conv = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=kernel_size)
        self.o_conv = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=kernel_size)

        self.conv_linear = nn.Linear(hidden_size, 3*hidden_size)
        self.rnn_linear = (2*hidden_size, hidden_size)
        
        self.w_xt = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_xf = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_xr = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.w_hy = nn.Parameter(torch.Tensor(input_size, hidden_size))
        
        self.b_x = nn.Parameter(torch.Tensor(hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))
        self.b_r = nn.Parameter(torch.Tensor(hidden_size))
        self.b_y = nn.Parameter(torch.Tensor(input_size))
        
    def forward(self, x, h, c):
        
        print("x",x.size())
        print("h",h.size())
        print("c",c.size())
        
        #squeeze as conv layer expects size [batch_size, input_size, length]
        x = x.unsqueeze(0)
        
        #CONVOLUTION STAGE
        
        z = F.tanh(self.z_conv(x)).squeeze(0)
        
        f = F.sigmoid(self.f_conv(x)).squeeze(0)
        
        o = F.sigmoid(self.o_conv(x)).squeeze(0)
        
        print("z",z.size())
        print("f",z.size())
        print("o",z.size())
        
        #POOL STAGE
       
        #fo-pooling
        
        c_ = (f*c.unsqueeze(1)) + (1-f)*z
        
        h = o * c_ 
        
        print("fo-pool c",c_.size())
        print("fo-pool h",h.size())      
        
        """t = torch.matmul(self.w_xt, x)
        
        f = F.sigmoid(torch.matmul(self.w_xf, x) + self.b_f)
        
        r = F.sigmoid(torch.matmul(self.w_xr, x) + self.b_r)

        c = f * c + (1 - f) * t

        h = r * F.tanh(c) + (1 - r) * x
        
        #y_t = w_hy * h_t + b_y
        y = torch.matmul(self.w_hy, h) + self.b_y"""
        
        print("END")
        
        return y, h, c

In [138]:
#x = T*n, where T is length of sequence and n is dimension of input
#z = convolutions across time step, with bank of m filters producing T*m
#filters of width k, each z_t only depends on x_{t-k-1} through x_t, done by padding to left size k-1
qrnn = QRNN(50, 100, 4)

x = Variable(torch.Tensor(50,13)) #size of 2 here as need to pad (kernel_size-1)
h = Variable(torch.Tensor(100))
c = Variable(torch.Tensor(100))

y, h, c = qrnn(x, h, c)

x torch.Size([50, 13])
h torch.Size([100])
c torch.Size([100])
z torch.Size([100, 10])
f torch.Size([100, 10])
o torch.Size([100, 10])
fo-pool c torch.Size([100, 10])
fo-pool h torch.Size([100, 10])
END
